In [67]:
import pandas as pd
import requests
import io
import altair as alt

In [17]:
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
r = requests.get(url)
data = r.content.decode('utf8')

In [32]:
url_states = "https://covidtracking.com/api/v1/states/daily.json"
r_states = requests.get(url_states)
states_data = r_states.content.decode('utf8')
states_df = pd.read_json(states_data)

In [77]:
class CovidApi:
    def __init__(self):
        url_states = "https://covidtracking.com/api/v1/states/daily.json"
        decoded = requests.get(url_states).content.decode('utf8')
        self.states_df = pd.read_json(decoded)
        self.states_df["date"] = self.states_df["date"].astype("str").astype("datetime64")
    
    @property
    def states(self):
        return set(self.states_df["state"])
    
    def filter_by(self, state = None, date = None):
        """
        Arg:
            date: formate yyyymmdd
        """
        data = self.states_df
        assert not state or state in self.states, f"state must be on of {self.states}"
        if state:
            data = data[data["state"] == state]
        if date:
            data = data[data["date"] == date]
        return data

In [78]:
covid_api = CovidApi()

In [79]:
alt.Chart(covid_api.filter_by(state="MA")).mark_bar().encode(
    x='date',
    y='deathIncrease'
)

alt.Chart(...)